In [9]:
%%writefile server.py

from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/')
def index():
    html = """
    <h1>안녕하세요<h1>
    """
    return html

datas = [45.7, 45, 10, 70.8]

@app.route('/signal')
def signal():
    global datas # global 변수인 것 명시
    datas.append( float(request.args.get("data")) )
    return str(datas)

@app.route('/view')
def view():
    global datas # global 변수인 것 명시
    
    labels = [i+1 for i in range(len(datas))]
                              
    ctx = {"title":"그래프^*^", "labels":labels, "data":datas}
    return render_template("view.html", ctx=ctx) # 반드시 ctx 딕셔너리로 보내기


if __name__== '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py


### template / view

In [ ]:

<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<meta http-equiv="refresh" content="10; view"
<head>
<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.8.0/Chart.bundle.min.js"></script>
<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.8.0/Chart.min.js"></script>
</head>

<body>
  <h1> {{ctx.title}} </h1>

  <canvas id="myChart" style="height:300; width:500"></canvas>

</body>
<script>

<!-- 파라미터 추가 -->
function addGraph(id, labels, data) {

var ctx = document.getElementById(id);
var myChart = new Chart(ctx, {
    type: 'line',
    data: {
        labels: labels,
        datasets: [{
            label: '그래프 테스트',
            data: data,
            borderWidth: 1
        }]
    },
    options: {
      <!-- 반응형 웹으로 하지 않겠다. -->
      responsive : true,
    }



});

}

<!-- 파이썬과 웹이 데이터를 주고받는 방식 -->
<!-- addGraph("myChart", [&#39;a&#39;, &#39;B&#39;, &#39;C&#39;], [101, 123, 78.2] ); -->
<!-- html 특수문자를 사용했을 때 변환되는 것을 -->
addGraph("myChart", {{ctx.labels|safe}}, {{ctx.data|safe}} );
//addGraph("myChart2", ['가', '나', '다', '라'], [12.7, 10, 65, 32] );


</script>


In [11]:
import requests

datas = [70.5, 60.7, 20.5, 30.4]
for data in datas:
    response = requests.get("http://localhost:8000/signal?data=" + str(data))
    # 동기방식으로 작동하기 떄문에 동시에 실행하려면 쓰레드로 별도 돌려야함
    
    print(response.text)

[45.7, 45, 10, 70.8, 7.5, 6.7, 0.5, 3.4, 70.5]
[45.7, 45, 10, 70.8, 7.5, 6.7, 0.5, 3.4, 70.5, 60.7]
[45.7, 45, 10, 70.8, 7.5, 6.7, 0.5, 3.4, 70.5, 60.7, 20.5]
[45.7, 45, 10, 70.8, 7.5, 6.7, 0.5, 3.4, 70.5, 60.7, 20.5, 30.4]


## 동영상 출력

In [33]:
%%writefile server.py

import cv2
from pyzbar.pyzbar import decode
from flask import Flask, request, render_template, make_response
app = Flask(__name__)

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')
    
@app.route('/')
def index():
    return  render_template("face.html")

@app.route('/upload', methods=["post"])
def upload():
    f = request.files['file1']   
    filename = "./static/" + f.filename
    f.save(filename)    
    
    img = cv2.imread(filename)
    if img.shape[0] >= 1000 and img.shape[1] >= 1000:
        img = cv2.resize(img, dsize=(0, 0), fx=0.25, fy=0.25)    

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(gray)
    
    if len(faces) > 0:
        for (x1, y1, w1, h1) in faces:
            cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 255), 2)
            faceROI = img[y1:y1 + h1, x1:x1 + w1]
            eyes = eye_classifier.detectMultiScale(faceROI)
            for (x2, y2, w2, h2) in eyes:
                    center = (int(x2 + w2 / 2), int(y2 + h2 / 2))
                    cv2.circle(img, center, int(w2 / 2), (255, 0, 0), 2, cv2.LINE_AA)

        retval, buffer = cv2.imencode('.jpg', img)
        response = make_response(buffer.tobytes())
        response.headers['Content-Type'] = 'image/jpg'
        return response

    code = decode(img)    
    url  = code[0].data.decode("utf-8")
        
    return f"<script> alert('이동합니다.'); window.location.href='{url}';</script>"

    
    
if __name__ == '__main__':
     app.run(host='0.0.0.0', debug=True, port=8000)      

Overwriting server.py


In [34]:
%%writefile server.py
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import io

from flask import Flask, request, render_template, make_response, Response
app = Flask(__name__)

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')
vc = cv2.VideoCapture(0)

@app.route('/')
def index():
    return  render_template("face.html")

@app.route('/upload', methods=["post"])
def upload():
    f = request.files['file1']   
    filename = "./static/" + f.filename
    f.save(filename)            
    #img = cv2.imdecode(np.fromstring(f.read(), np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.imread(filename)    
    #img = cv2.resize(img, dsize=(0, 0), fx=0.25, fy=0.25)        
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    
    if len(faces) > 0 :   
        for (x1, y1, w1, h1) in faces:
            cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 255), 2)

            faceROI = gray[y1:y1 + h1, x1:x1 + w1]
            cv2.imshow('face', faceROI)                

            eyes = eye_classifier.detectMultiScale(faceROI)
            for (x2, y2, w2, h2) in eyes:
                center = (int(x1+ x2 + w2 / 2), int(y1 +y2 + h2 / 2))
                cv2.circle(img, center, int(w2 / 2), (255, 0, 0), 2, cv2.LINE_AA)

        retval, buffer = cv2.imencode('.jpg', img)  


        response = make_response(buffer.tobytes())
        response.headers['Content-Type'] = 'image/jpg'

        return response    
    
    code = decode(img)    
    url  = code[0].data.decode("utf-8")    
    
    return f"<a href={url}>QR코드가기</a>" 
    
    #return  f"<script> alert('이동합니다.'); window.location.href='{url}'; </script>"
    
    
    
def gen():
    while True:
        read_return_code, frame = vc.read()
        encode_return_code, image_buffer = cv2.imencode('.jpg', frame)
        io_buf = io.BytesIO(image_buffer)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + io_buf.read() + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(
        gen(),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

@app.route('/test')
def test():
    return  "<h1>동영상 테스트</h1> <img src=/video_feed width=320 height=240>"



if __name__ == '__main__':
     app.run(host='0.0.0.0', debug=True, port=8000)      

Overwriting server.py


In [39]:
%%writefile server.py
import cv2
import numpy as np
import io
import random

from flask import Flask, request, render_template, make_response, Response
app = Flask(__name__)

vc = cv2.VideoCapture('vtest.avi')

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

datas = []

def detectHuman(frame) :
    detected, _ = hog.detectMultiScale(frame)

    for (x, y, w, h) in detected:
        c = (random.randint(0, 255),
             random.randint(0, 255), 
             random.randint(0, 255))
        cv2.rectangle(frame, (x, y), (x + w, y + h), c, 3)
    return detected    

@app.route('/')
def index():
    return  render_template("human.html")

def gen():
    global vc
    global datas
    
    while True:
        read_return_code, frame = vc.read()
        
        if not read_return_code :
            vc = cv2.VideoCapture('vtest.avi')
            datas = []
            break;
        
        rect = detectHuman(frame)
        
        datas.append(len(rect))
        print(datas)
        
        encode_return_code, image_buffer = cv2.imencode('.jpg', frame)
        io_buf = io.BytesIO(image_buffer)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + io_buf.read() + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(
        gen(),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

@app.route('/view')
def view():
    global datas
    return str(datas)
    
if __name__ == '__main__':
     app.run(host='0.0.0.0', debug=True, port=8000)      

Overwriting server.py


In [18]:
a = [2, 2, 1, 2, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2, 4, 4, 2, 4, 3, 3, 5, 4, 5, 3, 4, 4, 4, 2, 3, 3, 4, 2, 3, 4, 3, 1, 2, 3, 4, 3, 4, 2, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 5, 4, 6, 3, 3, 3, 3, 2, 2, 3, 3, 3, 4, 2, 2, 3, 3, 4, 2, 5, 4, 3, 3, 2, 3, 3, 3, 3, 3, 4, 4, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 1, 1, 3, 2, 0, 2, 2, 2, 2, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 3, 5, 2, 4, 2, 2, 2, 3, 1, 3, 2, 3, 2, 5, 2, 4, 1, 2, 2, 4, 5, 2, 3, 3, 2, 2, 3, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 2, 3, 3, 5, 3, 3, 4, 4, 3, 3, 5, 5, 4, 4, 4, 4, 3, 4, 4, 3, 3, 3, 2, 3, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 4, 2, 2, 2, 3, 3, 3, 4, 3, 3, 6, 5, 5, 6, 5, 5, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 3, 3]

In [26]:
a.count([0, 1])

0

In [29]:
 [a.count(i) for i in list(set(a))]

[1, 24, 93, 84, 40, 24, 3]

In [30]:
print(list(set(a)))

[0, 1, 2, 3, 4, 5, 6]
